In [ ]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd

project = 'nyc-yellow-trips' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [ ]:
# Function to execute a BigQuery query and return a DataFrame

def query_to_dataframe(query: str) -> pd.DataFrame:
    """
    Executes a SQL query in BigQuery and returns a Pandas DataFrame.

    Parameters:
    - query (str): The SQL query to execute.

    Return:
    - pd.DataFrame : The DataFrame containing the results of the query.
    """
    try:
        df = client.query(query).to_dataframe()
        print(f"Query executed successfully. Retrieved {df.shape[0]} rows.")
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return pd.DataFrame()

In [ ]:
query_trips_ml_data = """
SELECT *
FROM `nyc-yellow-trips.ml_dataset.trips_ml_data`
"""
trips_ml_data_df = query_to_dataframe(query_trips_ml_data)
trips_ml_data_df.head()

Query executed successfully. Retrieved 3097061 rows.


VendorID      tpep_pickup_datetime     tpep_dropoff_datetime  \
0         2 2024-11-12 17:36:01+00:00 2024-11-12 17:45:05+00:00   
1         2 2024-11-16 12:57:25+00:00 2024-11-16 12:57:35+00:00   
2         2 2024-11-09 00:29:27+00:00 2024-11-09 00:37:11+00:00   
3         2 2024-11-14 07:46:36+00:00 2024-11-14 07:47:45+00:00   
4         2 2024-11-22 11:01:27+00:00 2024-11-22 11:03:34+00:00   

   passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0              1.0           1.55         1.0                  N   
1              2.0           0.02         1.0                  N   
2              4.0           0.18         1.0                  N   
3              2.0           0.13         1.0                  N   
4              1.0           0.46         1.0                  N   

   PULocationID  DOLocationID  payment_type  fare_amount  extra  mta_tax  \
0           219           219             2          0.0    0.0      0.5   
1            43           236             2          0.0    0.0      0.5   
2            43           163             2          0.0    0.0      0.5   
3            95            95             2          3.7    0.0      0.5   
4           151            24             2          4.4    0.0      0.5   

   tip_amount  tolls_amount  improvement_surcharge  total_amount  \
0         0.0           0.0                    1.0           1.5   
1         0.0           0.0                    1.0           4.0   
2         0.0           0.0                    1.0           4.0   
3         0.0           0.0                    1.0           5.2   
4         0.0           0.0                    1.0           5.9   

   congestion_surcharge  airport_fee                      source_file  
0                   0.0          0.0  yellow_tripdata_2024-11.parquet  
1                   2.5          0.0  yellow_tripdata_2024-11.parquet  
2                   2.5          0.0  yellow_tripdata_2024-11.parquet  
3                   0.0          0.0  yellow_tripdata_2024-11.parquet  
4                   0.0          0.0  yellow_tripdata_2024-11.parquet

In [ ]:
trips_ml_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3097061 entries, 0 to 3097060
Data columns (total 20 columns):
 #   Column                 Dtype              
---  ------                 -----              
 0   VendorID               Int64              
 1   tpep_pickup_datetime   datetime64[us, UTC]
 2   tpep_dropoff_datetime  datetime64[us, UTC]
 3   passenger_count        float64            
 4   trip_distance          float64            
 5   RatecodeID             float64            
 6   store_and_fwd_flag     object             
 7   PULocationID           Int64              
 8   DOLocationID           Int64              
 9   payment_type           Int64              
 10  fare_amount            float64            
 11  extra                  float64            
 12  mta_tax                float64            
 13  tip_amount             float64            
 14  tolls_amount           float64            
 15  improvement_surcharge  float64            
 16  total_amount      

In [ ]:
# Missing values
trips_ml_data_df.isna().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
source_file              0
dtype: int64

In [ ]:
def preprocess_data(df):
    # Ensure datetime columns are in datetime format
    #df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
    #df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

    # Trip duration in minutes
    df["trip_duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

    # Extract time-based features
    df["pickup_dayofweek"] = df["tpep_pickup_datetime"].dt.dayofweek # Monday=0, Sunday=6.
    df["pickup_month"] = df["tpep_pickup_datetime"].dt.month
    df["pickup_year"] = df["tpep_pickup_datetime"].dt.year
    df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour
    df["is_weekend"] = df["pickup_dayofweek"].isin([5, 6]).astype(int)  # 5=Saturday, 6=Sunday

    # Filter payment type (Credit Card = 1, Cash = 2)
    #df = df[df["payment_type"].isin([1, 2])].copy()

    # Create binary feature for credit card payments
    df["is_credit_card"] = (df["payment_type"] == 1).astype(int)

    # Select relevant columns
    selected_cols = [
        "PULocationID", "DOLocationID", "passenger_count", "trip_distance",
        "trip_duration", "pickup_dayofweek", "pickup_month", "pickup_year", "pickup_hour",
        "is_weekend", "is_credit_card", "total_amount"
    ]

    return df[selected_cols].copy()


In [ ]:
from sklearn.model_selection import train_test_split

def split_data(df, train_size=0.7, val_size=0.15, test_size=0.15, random_state=42):
    """
    Splits the dataframe into train, validation, and test sets.

    Parameters:
    - df: Pandas DataFrame
    - train_size: Proportion of the dataset for training (default=70%)
    - val_size: Proportion for validation (default=15%)
    - test_size: Proportion for testing (default=15%)
    - random_state: Seed for reproducibility

    Returns:
    - train_df, val_df, test_df: Split DataFrames
    """
    assert train_size + val_size + test_size == 1, "Split sizes must sum to 1"

    # First, split train + val and test
    train_val_df, test_df = train_test_split(df, test_size=test_size, random_state=random_state)

    # Then, split train and validation
    train_df, val_df = train_test_split(train_val_df, test_size=val_size / (train_size + val_size),
                                        random_state=random_state)

    return train_df, val_df, test_df

# Apply the function
train_df, val_df, test_df = split_data(trips_ml_data_df)

# Display the sizes
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")


Train size: 2167941
Validation size: 464560
Test size: 464560


In [ ]:
preprocessed_train_df = preprocess_data(train_df)
preprocessed_train_df.head()

PULocationID  DOLocationID  passenger_count  trip_distance  \
2841151            50           170              1.0           2.59   
1836945           234           114              2.0           1.43   
1963711           234           211              1.0           1.20   
360510             70           211              1.0           9.90   
839470             87             4              5.0           2.63   

         trip_duration  pickup_dayofweek  pickup_month  pickup_year  \
2841151      15.633333                 5            11         2024   
1836945      12.500000                 5            11         2024   
1963711       8.366667                 4            11         2024   
360510       19.300000                 4            11         2024   
839470        5.883333                 1            11         2024   

         pickup_hour  is_weekend  is_credit_card  total_amount  
2841151            2           1               1         25.56  
1836945           23           1               1         19.80  
1963711           13           0               1         15.95  
360510            22           0               1         61.35  
839470            23           0               1         17.80

In [ ]:
preprocessed_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2167941 entries, 2841151 to 2971570
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   PULocationID      Int64  
 1   DOLocationID      Int64  
 2   passenger_count   float64
 3   trip_distance     float64
 4   trip_duration     float64
 5   pickup_dayofweek  int32  
 6   pickup_month      int32  
 7   pickup_year       int32  
 8   pickup_hour       int32  
 9   is_weekend        int64  
 10  is_credit_card    int64  
 11  total_amount      float64
dtypes: Int64(2), float64(4), int32(4), int64(2)
memory usage: 186.1 MB


In [ ]:
# Load the preprocessed_train_df dataframe into BigQuery

DATASET_ID = "ml_dataset"
TABLE_ID = "preprocessed_train_data"
FULL_TABLE_ID = f"{project}.{DATASET_ID}.{TABLE_ID}"

# Define schema (ensure correct types)
schema = [
    bigquery.SchemaField("PULocationID", "INTEGER"),
    bigquery.SchemaField("DOLocationID", "INTEGER"),
    bigquery.SchemaField("passenger_count", "FLOAT"),
    bigquery.SchemaField("trip_distance", "FLOAT"),
    bigquery.SchemaField("trip_duration", "FLOAT"),
    bigquery.SchemaField("pickup_dayofweek", "INTEGER"),
    bigquery.SchemaField("pickup_month", "INTEGER"),
    bigquery.SchemaField("pickup_year", "INTEGER"),
    bigquery.SchemaField("pickup_hour", "INTEGER"),
    bigquery.SchemaField("is_weekend", "INTEGER"),
    bigquery.SchemaField("is_credit_card", "INTEGER"),
    bigquery.SchemaField("total_amount", "FLOAT"),
]

# Load data into BigQuery
job = client.load_table_from_dataframe(
    preprocessed_train_df, FULL_TABLE_ID, job_config=bigquery.LoadJobConfig(schema=schema)
)

# Wait for the job to complete
job.result()

print(f"Data successfully uploaded to BigQuery: {FULL_TABLE_ID}")

Data successfully uploaded to BigQuery: nyc-yellow-trips.ml_dataset.preprocessed_train_data


In [19]:
preprocessed_test_df = preprocess_data(test_df)
preprocessed_test_df.head()

PULocationID  DOLocationID  passenger_count  trip_distance  \
1709461           170           229              1.0           0.35   
1003500           262            68              1.0           4.32   
585717            262           262              2.0           0.40   
1944669           107           162              1.0           0.62   
1713433            48           237              1.0           2.18   

         trip_duration  pickup_dayofweek  pickup_month  pickup_year  \
1709461       6.800000                 1            11         2024   
1003500      31.850000                 3            11         2024   
585717        4.483333                 3            11         2024   
1944669       6.333333                 3            11         2024   
1713433      14.600000                 4            11         2024   

         pickup_hour  is_weekend  is_credit_card  total_amount  
1709461           17           0               1         17.12  
1003500           15           0               1         42.88  
585717            15           0               0          9.10  
1944669            7           0               1         13.90  
1713433           16           0               1         25.68

In [20]:
preprocessed_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 464560 entries, 1709461 to 447969
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   PULocationID      464560 non-null  Int64  
 1   DOLocationID      464560 non-null  Int64  
 2   passenger_count   464560 non-null  float64
 3   trip_distance     464560 non-null  float64
 4   trip_duration     464560 non-null  float64
 5   pickup_dayofweek  464560 non-null  int32  
 6   pickup_month      464560 non-null  int32  
 7   pickup_year       464560 non-null  int32  
 8   pickup_hour       464560 non-null  int32  
 9   is_weekend        464560 non-null  int64  
 10  is_credit_card    464560 non-null  int64  
 11  total_amount      464560 non-null  float64
dtypes: Int64(2), float64(4), int32(4), int64(2)
memory usage: 39.9 MB


In [21]:
preprocessed_test_df.shape

(464560, 12)

In [22]:
# Load the preprocessed_test_df dataframe into BigQuery

DATASET_ID = "ml_dataset"
TABLE_ID = "preprocessed_test_data"
FULL_TABLE_ID = f"{project}.{DATASET_ID}.{TABLE_ID}"

# Define schema (ensure correct types)
schema = [
    bigquery.SchemaField("PULocationID", "INTEGER"),
    bigquery.SchemaField("DOLocationID", "INTEGER"),
    bigquery.SchemaField("passenger_count", "FLOAT"),
    bigquery.SchemaField("trip_distance", "FLOAT"),
    bigquery.SchemaField("trip_duration", "FLOAT"),
    bigquery.SchemaField("pickup_dayofweek", "INTEGER"),
    bigquery.SchemaField("pickup_month", "INTEGER"),
    bigquery.SchemaField("pickup_year", "INTEGER"),
    bigquery.SchemaField("pickup_hour", "INTEGER"),
    bigquery.SchemaField("is_weekend", "INTEGER"),
    bigquery.SchemaField("is_credit_card", "INTEGER"),
    bigquery.SchemaField("total_amount", "FLOAT"),
]

# Load data into BigQuery
job = client.load_table_from_dataframe(
    preprocessed_test_df, FULL_TABLE_ID, job_config=bigquery.LoadJobConfig(schema=schema)
)

# Wait for the job to complete
job.result()

print(f"Data successfully uploaded to BigQuery: {FULL_TABLE_ID}")

Data successfully uploaded to BigQuery: nyc-yellow-trips.ml_dataset.preprocessed_test_data


In [ ]:
# You can continue to create a custom model
